Agent Assistant to Retrieve Data from Wikipidea.

In [1]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

load_dotenv()

True

In [2]:
current_dir = os.path.curdir
persistence_directory = os.path.join(current_dir, "db", "assassins_db")
embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1")

if not os.path.exists(persistence_directory):
    print("persistent directory does not exist, Initializing vector store...")

    urls = [
        "https://en.wikipedia.org/wiki/Assassin%27s_Creed"
    ]
    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]
    print(docs_list)

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
    soc_splits = text_splitter.split_documents(docs_list)

    db = Chroma.from_documents(
        documents=docs_list,
        collection_name="rag-chroma",
        embedding=embedding,
        persist_directory=persistence_directory
    )
else:
    db = Chroma(persist_directory=persistence_directory, embedding_function=embedding)

persistent directory does not exist, Initializing vector store...
[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Assassin%27s_Creed', 'title': "Assassin's Creed - Wikipedia", 'language': 'en'}, page_content='\n\n\n\nAssassin\'s Creed - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [3]:
retriever = db.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [4]:
def retrieve(query:str):
    relevant_docs = retriever.invoke(query)
    return relevant_docs

Integrate with LLM

In [5]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

In [6]:
llm = ChatOllama(model="llama3", temperature=0, verbose=1)


contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

contextualize_q__prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q__prompt)

qa_system_prompt = """
You are an assistant for question-answering tasks about Assassins's Creed.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in this format:
1. Start with "Based on Assassins's Wekipedia:" if relevant
2. End with "Ask me another question about Assassins's Creed!"
Use sentences to make you answer concise but not too long.
Context: {context} 
Answer:"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

chat_history = []  # Collect chat history here (a sequence of messages)
# Function to simulate a continual chat with calling not chatting
def continual_chat(query):
    print("Start chatting")
    # Process the user's query through the retrieval chain
    result = rag_chain.invoke({"input": query, "chat_history": chat_history})
    # Display the AI's response
    print(f"AI: {result['answer']}")
    # Update the chat history
    chat_history.append(HumanMessage(content=query))
    chat_history.append(SystemMessage(content=result["answer"]))


In [7]:
continual_chat("what is the second series of assassins creed?")

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Start chatting
AI: The second game in the Assassin's Creed series is titled "Assassin's Creed II". It was released in 2009 for PlayStation 3, Xbox 360, and Microsoft Windows.

Assassin's Creed II takes place in the late 15th century, during the Italian Renaissance. The game follows Ezio Auditore da Firenze, a new protagonist who is a member of the Assassin Brotherhood. The story takes place in Italy, specifically in Florence, Venice, Forli, and Rome.

The game's storyline revolves around Ezio's quest for revenge against the Templar Order, which he believes is responsible for his family's death. Along the way, he uncovers a deeper conspiracy involving the Templars' plans to control the world.

Assassin's Creed II introduced several new features to the series, including:

1. Improved parkour mechanics: Ezio can climb and jump more efficiently than Altaïr in the first game.
2. New combat mechanics: Ezio has access to a variety of weapons and abilities, such as dual-wielding pistols and th

In [8]:
continual_chat("what is the full name of its main character?")

Start chatting


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


AI: The main character in Assassin's Creed is Ezio Auditore da Firenze. His full name is Ezio Alessandro Auditore Orsini da Firenze.


In [9]:
continual_chat("Nice, did he died in this series?, and if he tell me how")

Start chatting


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


AI: Ezio Auditore da Firenze, the protagonist of Assassin's Creed II, does indeed die at the end of his story arc.

Spoiler alert!

At the end of Assassin's Creed II, Ezio is shown to have grown old and wise. He has fulfilled his revenge against the Templars and has become a legendary figure in the Assassin Brotherhood. As he reflects on his life, he realizes that it's time for him to pass on the mantle to the next generation.

In a poignant scene, Ezio is shown to be sitting on a rooftop, looking out at the city of Rome. He takes a deep breath, and as he exhales, his eyes glaze over, and he passes away, surrounded by the memories of his loved ones and the Assassin Brotherhood.

The game's ending is intentionally left open-ended, leaving players wondering about the fate of Ezio's legacy and the future of the Assassin Brotherhood.
